In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc


import sys
import seaborn as sns
import celloracle as co
co.__version__


from celloracle.applications import Pseudotime_calculator

In [ ]:
# visualization settings
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.rcParams['figure.figsize'] = [4.5, 4.5]
plt.rcParams["savefig.dpi"] = 300

In [ ]:
# reload dataset
adata = sc.read_h5ad('./cornea_limbus_epithelium_count.h5ad')

In [ ]:

adata=adata[:,highly_variable]



In [ ]:
sc.pp.normalize_per_cell(adata)

In [ ]:
# keep raw cont data before log transformation

adata.raw=adata
adata.layers["raw_count"] = adata.raw.X.copy()


# Log transformation and scaling
sc.pp.log1p(adata)
sc.pp.scale(adata)

In [ ]:
# PCA
sc.tl.pca(adata, svd_solver='arpack')
sc.pl.pca_variance_ratio(adata, log=True)



In [ ]:
sc.pp.neighbors(adata, n_neighbors=4, n_pcs=20)



In [ ]:

import bbknn  
sc.external.pp.bbknn(adata, batch_key='orig.ident')


In [ ]:
#sc.tl.umap(adata, init_pos='paga')
sc.tl.umap(adata)
sc.tl.leiden(adata,resolution =0.8)
sc.pl.umap(adata, color=['group','leiden'],legend_fontsize='large',legend_loc='on data')

In [ ]:

cluster_list = adata.obs.leiden.unique()
# Make cluster anottation dictionary
annotation = {"E_LSC": [10],
              "E_cornea.basal":[0],
              "E_cornea.suprabasal":[6],  
              "E_cornea.superficial":[2],
              "E_limbal.superficial":[9],
              "adult_LSC": [7],
              "adult_cornea.basal":[8],
              "adult_cornea.suprabasal":[1],
              "adult_limbal.suprabasal" :[5],
              "adult_cornea.superficial":[4],
              "adult_limbal.superficial":[3]}
             

# Change dictionary format
annotation_rev = {}
for i in cluster_list:
    for k in annotation:
        if int(i) in annotation[k]:
            annotation_rev[i] = k

# Check dictionary
annotation_rev

adata.obs["leiden_anno"] = [annotation_rev[i] for i in adata.obs.leiden]

In [ ]:
sc.pl.umap(adata, color='leiden_anno',legend_fontsize='large',frameon=False,save="cornea.epithelium.umap.pdf")

In [ ]:
sc.pl.umap(adata, color=['group'], frameon=False,legend_fontsize='large',palette='Paired_r',save="cornea.epithelium.group.umap.pdf")


In [ ]:
adata.write_h5ad("cornea_limbus_epithelium.highly_variable_scanpy.h5ad")

In [ ]:
adata = sc.read_h5ad('cornea_limbus_epithelium.highly_variable_scanpy.h5ad')

In [ ]:
# Instantiate Oracle object
oracle = co.Oracle()

In [ ]:
# Check data in anndata
print("Metadata columns :", list(adata.obs.columns))
print("Dimensional reduction: ", list(adata.obsm.keys()))

In [ ]:
adata

In [ ]:
# In this notebook, we use the unscaled mRNA count for the nput of Oracle object.
adata.X = adata.layers["raw_count"].copy()

# Instantiate Oracle object.
oracle.import_anndata_as_raw_count(adata=adata,
                                   cluster_column_name='leiden_anno',#"cell_type",
                                   embedding_name='X_umap')#"X_draw_graph_fa")




In [ ]:
# Load the TF and target gene information from Paul et al. (2015).
regulons = pd.read_csv("cornea_epithelium.count.regulons_celloracle.csv")


regulons=regulons.astype(str)
regulons


In [ ]:
# Make dictionary: dictionary key is TF and dictionary value is list of target genes.
TF_to_TG_dictionary = {}

for TF, TGs in zip(regulons.TF, regulons.Target_genes):
    # convert target gene to list
    TG_list = TGs.replace(" ", "").split(",")
    # store target gene list in a dictionary
    TF_to_TG_dictionary[TF] = TG_list

# We invert the dictionary above using a utility function in celloracle.
TG_to_TF_dictionary = co.utility.inverse_dictionary(TF_to_TG_dictionary)

In [ ]:

oracle.addTFinfo_dictionary(TG_to_TF_dictionary)

In [ ]:
# Perform PCA
oracle.perform_PCA()

# Select important PCs
plt.plot(np.cumsum(oracle.pca.explained_variance_ratio_)[:100])
n_comps = np.where(np.diff(np.diff(np.cumsum(oracle.pca.explained_variance_ratio_))>0.002))[0][0]
plt.axvline(n_comps, c="k")
plt.show()
print(n_comps)
n_comps = min(n_comps, 50)

In [ ]:
n_cell = oracle.adata.shape[0]
print(f"cell number is :{n_cell}")

In [ ]:
k = int(0.025*n_cell)
print(f"Auto-selected k is :{k}")

In [ ]:

oracle.knn_imputation(n_pca_dims=n_comps, k=k, balanced=True, b_sight=k*8,
                      b_maxl=k*4, n_jobs=24)

In [ ]:
# Save oracle object.
oracle.to_hdf5("cornea_epithlium.highly_variable_final.celloracle.oracle")

In [ ]:
# Load file.
oracle = co.load_hdf5("cornea_epithlium.highly_variable_final.celloracle.oracle")

In [ ]:
###GRN calculation
# Check clustering data
#sc.pl.draw_graph(oracle.adata, color=["cell_type","leiden_anno"])

sc.pl.umap(adata, color='leiden_anno',legend_fontsize='large')

In [ ]:
%%time
# Calculate GRN for each population in "louvain_annot" clustering unit.
# This step may take some time.(~30 minutes)
links = oracle.get_links(cluster_name_for_GRN_unit="leiden_anno", alpha=10,
                         verbose_level=10, n_jobs=48)

In [ ]:
#order = ['6', '11', '9','2', '8', '7', '3', '5', '4', '0', '1', '10']
       
  
order = ["E_LSC",
"E_cornea.basal",
"E_cornea.suprabasal",  
"E_cornea.superficial",
"E_limbal.superficial",
"adult_LSC",
"adult_cornea.basal",
"adult_cornea.suprabasal",
"adult_limbal.suprabasal" ,
"adult_cornea.superficial",
"adult_limbal.superficial"]
         
links.palette = links.palette.loc[order]

In [ ]:
#Network preprocessing
links.filter_links(p=0.001, weight="coef_abs", threshold_number=2000)

In [ ]:
plt.rcParams["figure.figsize"] = [9, 4.5]
links.plot_degree_distributions(plot_model=True,
                                               #save=f"{save_folder}/degree_distribution/",
                                               )

In [ ]:
plt.rcParams["figure.figsize"] = [6, 8]

In [ ]:
# Calculate network scores.
links.get_network_score()

In [ ]:
links.merged_score.head()

In [ ]:
# 保存Save Links object.
links.to_hdf5(file_path="links_final.celloracle.links")

In [ ]:
# 读取You can load files with the following command.
links = co.load_hdf5(file_path="links_final.celloracle.links")

In [ ]:
# Check cluster name
links.cluster

In [ ]:
# Visualize top n-th genes with high scores.
links.plot_scores_as_rank(cluster="adult_cornea.superficial", n_gene=50, save=f"{save_folder}/ranked_score")


In [ ]:
# Compare GRN score between two clusters
links.plot_score_comparison_2D(value="eigenvector_centrality",
                               cluster1="adult_LSC", cluster2="adult_cornea.superficial",
                               percentile=98,
                               save=f"{save_folder}/score_comparison")

In [ ]:
# Compare GRN score between two clusters
links.plot_score_comparison_2D(value="betweenness_centrality",
                                cluster1="adult_LSC", cluster2="adult_cornea.superficial",
                               percentile=98,
                               save=f"{save_folder}/score_comparison")

In [ ]:
# Compare GRN score between two clusters
links.plot_score_comparison_2D(value="degree_centrality_all",
                                cluster1="adult_LSC", cluster2="adult_cornea.superficial",
                               percentile=98, save=f"{save_folder}/score_comparison")

In [ ]:
# Visualize Gata2 network score dynamics
links.plot_score_per_cluster(goi="RORA", save=f"{save_folder}/network_score_per_gene/")

In [ ]:
# Visualize Cebpa network score dynamics
links.plot_score_per_cluster(goi="PITX1")

In [ ]:
cluster_name = "adult_cornea.superficial"
filtered_links_df = links.filtered_links[cluster_name]
filtered_links_df.head()

In [ ]:
filtered_links_df[filtered_links_df.source == "RORA"]

In [ ]:
plt.rcParams["figure.figsize"] = [6, 4.5]

In [ ]:
# Plot degree_centrality
plt.subplots_adjust(left=0.15, bottom=0.3)
plt.ylim([0,0.040])
links.plot_score_discributions(values=["degree_centrality_all"],
                               method="boxplot",
                               save=f"{save_folder}",
                              )



In [ ]:
# Plot eigenvector_centrality
plt.subplots_adjust(left=0.15, bottom=0.3)
plt.ylim([0, 0.28])
links.plot_score_discributions(values=["eigenvector_centrality"],
                               method="boxplot",
                               save=f"{save_folder}")


In [ ]:
plt.subplots_adjust(left=0.15, bottom=0.3)
links.plot_network_entropy_distributions(save=f"{save_folder}")



In [ ]:
links.filter_links()
oracle.get_cluster_specific_TFdict_from_Links(links_object=links)
oracle.fit_GRN_for_simulation(alpha=10,
                              use_cluster_specific_TFdict=True)

In [ ]:
# Check gene expression
goi = "PITX1"

#sc.pl.umap(oracle.adata, color=[goi, oracle.cluster_column_name],
 #                layer="imputed_count",vmax="q99", use_raw=False,save="PITX1_umap.pdf")

In [ ]:
# Plot gene expression in histogram
sc.get.obs_df(oracle.adata, keys=[goi], layer="imputed_count").hist()
plt.show()

In [ ]:
# Enter perturbation conditions to simulate signal propagation after the perturbation.
oracle.simulate_shift(perturb_condition={goi: 0.0},
                      n_propagation=3)

In [ ]:
# Get transition probability
oracle.estimate_transition_prob(n_neighbors=200,
                                knn_random=True,
                                sampled_fraction=1,n_jobs=48)

# Calculate embedding
oracle.calculate_embedding_shift(sigma_corr=0.05)

In [ ]:
fig, ax = plt.subplots(1, 2,  figsize=[13, 6])

scale =20
# Show quiver plot
oracle.plot_quiver(scale=scale, ax=ax[0])
ax[0].set_title(f"Simulated cell identity shift vector: {goi} KO")

# Show quiver plot that was calculated with randomized graph.
oracle.plot_quiver_random(scale=scale, ax=ax[1])
ax[1].set_title(f"Randomized simulation vector")

plt.show()

In [ ]:
# n_grid = 40 is a good starting value.
n_grid = 40
oracle.calculate_p_mass(smooth=0.8, n_grid=n_grid, n_neighbors=200)

In [ ]:
# Search for best min_mass.
oracle.suggest_mass_thresholds(n_suggestion=12)

In [ ]:
min_mass = 24
oracle.calculate_mass_filter(min_mass=min_mass, plot=True)

In [ ]:
fig, ax = plt.subplots(1, 2,  figsize=[13, 6])

scale_simulation = 1.3 ##
# Show quiver plot
oracle.plot_simulation_flow_on_grid(scale=scale_simulation, ax=ax[0])
ax[0].set_title(f"Simulated cell identity shift vector: {goi} KO")

# Show quiver plot that was calculated with randomized graph.
oracle.plot_simulation_flow_random_on_grid(scale=scale_simulation, ax=ax[1])
ax[1].set_title(f"Randomized simulation vector")

plt.show()

In [ ]:
# Plot vector field with cell cluster
fig, ax = plt.subplots(figsize=[5, 5])

oracle.plot_cluster_whole(ax=ax, s=5)
oracle.plot_simulation_flow_on_grid(scale=scale_simulation, ax=ax, show_background=False)

plt.savefig("./figures/PITX1 KO flow_final.pdf")


In [ ]:
# Instantiate pseudotime object using oracle object.
pt = Pseudotime_calculator(oracle_object=oracle)

In [ ]:
###
cluster_column_name='leiden_anno'
print("Clustering name: ", pt.cluster_column_name)
print("Cluster list", pt.cluster_list)

In [ ]:
# Check data
pt.plot_cluster(fontsize=5)

In [ ]:
# Here, clusters can be classified into either MEP lineage or GMP lineage


clusters_in_PCW_lineage = ["E_LSC",
"E_cornea.basal",
"E_cornea.suprabasal",  
"E_cornea.superficial",
"E_limbal.superficial"]


clusters_in_adult_lineage = ["adult_LSC",
"adult_cornea.basal",
"adult_cornea.suprabasal",
"adult_limbal.suprabasal" ,
"adult_cornea.superficial",
"adult_limbal.superficial"]

#clusters_in_PCW_lineage = ['6', '11', '9','2', '8']
#clusters_in_adult_lineage = ['7', '3', '5', '4', '0', '1', '10']


#clusters_in_PCW_lineage = ['limbal.suprabasal', 'limbal.superfcial']
#clusters_in_adult_lineage = ['LSC', 'cornea.basal',"cornea.suprabasal", 'cornea.superficial','limbal.suprabasal', 'limbal.superficial']

# Make a dictionary
lineage_dictionary = {"Lineage_PCW": clusters_in_PCW_lineage,
           "Lineage_adult": clusters_in_adult_lineage}

# Input lineage information into pseudotime object
pt.set_lineage(lineage_dictionary=lineage_dictionary)

# Visualize lineage information
pt.plot_lineages()

In [ ]:
# Show interactive plot using plotly. Please make sure that plotly is installed.

try:
    import plotly.express as px
    def plot(adata, embedding_key, cluster_column_name):
        embedding = adata.obsm[embedding_key]
        df = pd.DataFrame(embedding, columns=["x", "y"])
        df["cluster"] = adata.obs[cluster_column_name].values
        df["label"] = adata.obs.index.values
        fig = px.scatter(df, x="x", y="y", hover_name=df["label"], color="cluster")
        fig.show()

    plot(adata=pt.adata,
         embedding_key=pt.obsm_key,
         cluster_column_name=pt.cluster_column_name)
except:
    print("Plotly not found in your environment. Did you install plotly? Please read the instruction above.")


In [ ]:
# Estimated root cell name for each lineage
#root_cells = {"Lineage_cornea": "adult.1_GTCTACCCAGGCATGA-1"}
#root_cells = {"Lineage_cornea": "adult.3_CGTTAGAAGTAGGAAG-1"}
#root_cells = {"Lineage_cornea": "PCW10.2_CTCAACCGTAACGTTC-1"}
#root_cells = {"Lineage_adult": "adult.3_CGGGACTCACGACGTC-1","Lineage_PCW": "PCW13.2_AACAACCTCTCAGAAC-1"}
root_cells = {"Lineage_adult": "adult.3_CGGGACTCACGACGTC-1","Lineage_PCW": "PCW16.2_TGTCCACAGATTAGAC-1"}

pt.set_root_cells(root_cells=root_cells)
# Check root cell and lineage
pt.plot_root_cells()

In [ ]:
# Check diffusion map data.
"X_diffmap" in pt.adata.obsm

In [ ]:
#plt.rcParams["font.family"] = "arial"
plt.rcParams["figure.figsize"] = [5,5]
%config InlineBackend.figure_format = 'retina'
plt.rcParams["savefig.dpi"] = 300

%matplotlib inline

In [ ]:
# Calculate pseudotime
plt.figure()
pt.get_pseudotime_per_each_lineage()
# Check results
pt.plot_pseudotime(cmap="rainbow")



In [ ]:
# Check result
pt.adata.obs[["Pseudotime"]].head()

In [ ]:
# Add calculated pseudotime data to the oracle object
oracle.adata.obs = pt.adata.obs

# Save updated oracle object
#oracle.to_hdf5(FILE_PATH)

In [ ]:
# Visualize pseudotime
fig, ax = plt.subplots(figsize=[6,5])

sc.pl.embedding(adata=oracle.adata, basis=oracle.embedding_name, ax=ax, cmap="rainbow",
                color=["Pseudotime"],save="cornea_epithelium_pseudotime_final.pdf")

In [ ]:
from celloracle.applications import Gradient_calculator

# Instantiate Gradient calculator object
gradient = Gradient_calculator(oracle_object=oracle, pseudotime_key="Pseudotime")

In [ ]:
gradient.calculate_p_mass(smooth=0.8, n_grid=n_grid, n_neighbors=200)
gradient.calculate_mass_filter(min_mass=min_mass, plot=True)

In [ ]:
gradient.transfer_data_into_grid(args={"method": "polynomial", "n_poly":3}, plot=True)

In [ ]:
# Calculate graddient
gradient.calculate_gradient()

# Show results
scale_dev = 20    ###
gradient.visualize_results(scale=scale_dev, s=5)

#plt.savefig("./figures/all_pseudotime_final.pdf") 

In [ ]:
# Visualize results
fig, ax = plt.subplots(figsize=[5, 5])
oracle.plot_cluster_whole(ax=ax, s=5)
gradient.plot_dev_flow_on_grid(scale=scale_dev, ax=ax,show_background=False)

#plt.savefig("./figures/WT_flow_final.pdf")


In [ ]:

# Save gradient object if you want.
gradient.to_hdf5("cornea_PITX1.celloracle.gradient")

In [ ]:
from celloracle.applications import Oracle_development_module

# Make Oracle_development_module to compare two vector field
dev = Oracle_development_module()

# Load development flow
dev.load_differentiation_reference_data(gradient_object=gradient)

# Load simulation result
dev.load_perturb_simulation_data(oracle_object=oracle)


# Calculate inner produc scores
dev.calculate_inner_product()
dev.calculate_digitized_ip(n_bins=10)

In [ ]:
# Show perturbation scores
vm = 0.1   ####

fig, ax = plt.subplots(1, 2, figsize=[12, 6])
dev.plot_inner_product_on_grid(vm=0.02, s=50, ax=ax[0])
ax[0].set_title(f"PS")

dev.plot_inner_product_random_on_grid(vm=vm, s=50, ax=ax[1])
ax[1].set_title(f"PS calculated with Randomized simulation vector")
plt.show()

In [ ]:
# Show perturbation scores with perturbation simulation vector field
fig, ax = plt.subplots(figsize=[6, 6])
dev.plot_inner_product_on_grid(vm=vm, s=50, ax=ax)
dev.plot_simulation_flow_on_grid(scale=scale_simulation, show_background=False, ax=ax)

plt.savefig("./figures/PITX1 KO flow_on_grid_final.pdf")

In [ ]:
# Let's visualize the results
dev.visualize_development_module_layout_0(s=5,
                                          scale_for_simulation=scale_simulation,
                                          s_grid=50,
                                          scale_for_pseudotime=scale_dev,
                                          vm=vm)


plt.savefig("./figures/PITX1 KO flow_all_final.pdf")

In [ ]:

cord=pd.DataFrame(data=adata.obsm['X_umap'],index=adata.obs_names,columns=['UMAP_1','UMAP_2'])
cord.to_csv('cornea_scanpy_X_umap.tsv',sep="\t") 

In [ ]:
meta=pd.DataFrame(data=adata.obs)
meta.to_csv('cornea_scanpy_metadata.tsv',sep="\t") 